In [1]:
import sys
import os
import keras_tuner
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Layer, Dropout
from tensorflow.keras import layers
from tensorflow.keras.layers import PReLU


# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("transformer0.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from FNN1_1 import baseline_deviation, baeline_out_deviation, baseline_long_deviation, baseline_relError, absSum
baseline_out_deviation = baeline_out_deviation

from GetXY import x_train, y_train, x_val, y_val, x_test, y_test, out_x_test, out_y_test, long_x_test, long_y_test, outsideExpr, absSum

2025-10-19 18:01:51.606523: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-19 18:01:57.125447: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-19 18:01:57.125776: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-19 18:01:57.128269: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-19 18:01:57.128454: I external/local_xla/xla/stream_executor

Epoch 1/200


I0000 00:00:1760889721.474794   85144 service.cc:145] XLA service 0xfffe380042e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760889721.474894   85144 service.cc:153]   StreamExecutor device (0): Orin, Compute Capability 8.7
2025-10-19 18:02:01.522696: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-19 18:02:01.838767: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 90300


22/60 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 24.8777

I0000 00:00:1760889723.610347   85144 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 21.1420 - val_loss: 17.7119
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 15.3701 - val_loss: 15.4855
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 13.3012 - val_loss: 13.1699
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10.7504 - val_loss: 10.2007
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7.7666 - val_loss: 6.7574
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.6986 - val_loss: 3.6933
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.4860 - val_loss: 1.9331
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.4716 - val_loss: 1.3152
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.1138 - val_loss: 1.0690
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9309 - val_loss: 0.9155
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8028 - val_loss: 0.8069
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6998

/home/capybara/Desktop/matura_project_python/github/matura/FNN1_1.py:246: RuntimeWarning: divide by zero encountered in divide
  relativeError = np.where(np.array(y_test) != 0, deviation.flatten() / np.abs(np.array(y_test)), deviation.flatten())


2 + -2 - 2
2543
-2.0

Expressions not in x:
-4 - -4 - -2
True
1457
2.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]


In [2]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='val_loss',
    mode = "min"
)

In [3]:
#defining the positional encoder modelled after the formula in the paper that was cited. (generated by gemini)
def posEncoding(max_seq_len, d_model):
    # Create a matrix of angles according to the formula
    angle_rads = get_angles(np.arange(max_seq_len)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
    
    # Apply sine to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    
    # Apply cosine to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
    # Add a batch dimension
    pos_encoding = angle_rads[np.newaxis, ...]
    
    return tf.cast(pos_encoding, dtype=tf.float32)

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

In [4]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [5]:
#edited by gemini
input_shape = x_train[0].shape
from tensorflow.keras.layers import Input, Flatten, Dense, PReLU, Dropout, Add, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# --- Updated build_model function ---
def build_model(hp, input_shape):
    K.clear_session()
    
    # Hyperparameters
    num_neurons = hp.Int("num_neurons", 1, 512)
    num_layers = hp.Int("num_layers", 1, 16)
    dropoutTF = hp.Boolean("dropoutT/F")

    # --- Model Architecture ---
    inputs = Input(shape=input_shape) # Expects shape like (15,)

    # The input shape is (batch, 15). We need to treat it as a sequence.
    # Reshape to (batch, 15, 1) to represent a sequence of 15 steps with 1 feature each.
    # This is the key step to make shapes compatible.
    sequence_length = input_shape[0]
    d_model = 1
    x = Reshape((sequence_length, d_model))(inputs)

    # Add positional encoding
    positional_encoding = posEncoding(sequence_length, d_model)
    x = Add()([x, positional_encoding])
    
    # Flatten the sequence back into a vector of shape (batch, 15) for the Dense layers.
    x = Flatten()(x)

    # Add the dense layers as before
    for i in range(num_layers):
        x = Dense(num_neurons)(x)
        x = PReLU()(x)
        if dropoutTF:
            x = Dropout(0.1)(x)

    # Output Layer
    outputs = Dense(1, activation='linear')(x)

    # Create and compile the model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="mse")

    return model
build_model(keras_tuner.HyperParameters(), input_shape)

<Functional name=functional, built=True>

In [6]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=lambda hp: build_model(hp, input_shape),    objective="val_loss",
    max_trials=50,
    executions_per_trial=1,
    overwrite=True,
    directory="FNN3Tuner",
    project_name="tuner3",
)

In [7]:
num_epochs = 100
tuner.search(train_dataset, epochs = num_epochs, validation_data = (val_dataset), verbose = 1, callbacks = [])

Trial 50 Complete [00h 02m 46s]
val_loss: 0.004658530931919813

Best val_loss So Far: 0.0013289122143760324
Total elapsed time: 01h 42m 05s


In [8]:
best_hps = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hps, input_shape)
best_model.summary()
tuner.results_summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 15)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 15, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add (Add)                       │ (1, 15, 1)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (1, 15)                │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 494)               │         7,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu (PReLU)                 │ (1, 494)               │           494 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (1, 1)                 │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,893 (34.74 KB)

 Trainable params: 8,893 (34.74 KB)

 Non-trainable params: 0 (0.00 B)

Results summary
Results in FNN3Tuner/tuner3
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 14 summary
Hyperparameters:
num_neurons: 494
num_layers: 1
dropoutT/F: False
Score: 0.0013289122143760324

Trial 21 summary
Hyperparameters:
num_neurons: 349
num_layers: 1
dropoutT/F: False
Score: 0.0013864522334188223

Trial 35 summary
Hyperparameters:
num_neurons: 344
num_layers: 1
dropoutT/F: False
Score: 0.0014143552398309112

Trial 39 summary
Hyperparameters:
num_neurons: 155
num_layers: 1
dropoutT/F: False
Score: 0.0027343942783772945

Trial 42 summary
Hyperparameters:
num_neurons: 147
num_layers: 1
dropoutT/F: False
Score: 0.002782547613605857

Trial 19 summary
Hyperparameters:
num_neurons: 512
num_layers: 11
dropoutT/F: False
Score: 0.002827545627951622

Trial 25 summary
Hyperparameters:
num_neurons: 138
num_layers: 1
dropoutT/F: False
Score: 0.002850163960829377

Trial 36 summary
Hyperparameters:
num_neurons: 435
num_layers: 12
dropoutT/F: False
Score: 0.003326

In [9]:
x_test_dataset = tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size)
out_x_test_dataset = tf.data.Dataset.from_tensor_slices(out_x_test).batch(batch_size)
long_x_test_dataset = tf.data.Dataset.from_tensor_slices(long_x_test).batch(batch_size)

In [10]:
best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 114ms/step - loss: 17.9630 - val_loss: 16.0346
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 14.8677 - val_loss: 13.9570
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 12.4577 - val_loss: 11.2111
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.2588 - val_loss: 7.7508
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.7976 - val_loss: 4.7690
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.1502 - val_loss: 2.2696
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.5822 - val_loss: 1.2129
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9063 - val_loss: 0.8345
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6043 - val_loss: 0.5674
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4086 - val_loss: 0.4025
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2880 - val_loss: 0.2980
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - 

In [11]:
predsInRange = best_model.predict(x_test_dataset)
predsOutRange = best_model.predict(out_x_test_dataset)
predsLongRange = best_model.predict(long_x_test_dataset)

46/46 ━━━━━━━━━━━━━━━━━━━━ 9s 131ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [12]:
import numpy as np

reldiffInRange = []
diffInRange = []
safe_y_test = np.where(np.isclose(y_test,0.0), 1.0, y_test)

for i in range(len(y_test)):
    diffInRange.append(abs(y_test[i] - predsInRange[i]))
    reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
print(len(diffInRange))
print("MAE in Range: ", np.mean(diffInRange))
print("MRE in Range: ", np.mean(reldiffInRange))

diffLongRange = []
for i in range(200, 300):
    diffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsInRange[i])))
    
NEEDdiffLongRange = []
for i in range(len(long_y_test)):
    NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsInRange[i])))
print("MAE longer Expressions: ", np.mean(NEEDdiffLongRange))

diffOutRange = []
for i in range(len(out_y_test)):
    diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
diff_out_relError = []
for i in range(len(out_y_test)):
    diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
print("MAE out Range: ", np.mean(diffOutRange))
print("MRE out Range: ", np.mean(diff_out_relError))

1457
MAE in Range:  0.034600347
MRE in Range:  0.013437587
MAE longer Expressions:  6.1168723
MAE out Range:  2.5046074
MRE out Range:  0.2908847


In [13]:
placeholder = absSum(outsideExpr)
diffOutRange = []
indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 

for i in indices_with_placeholder_22:
    diffOutRange.append(np.abs(out_y_test[i]-predsOutRange[i]))


In [14]:
meanDiff_InRange = np.mean(diffInRange)
meanDiff_OutRange = np.mean(diffOutRange)
meanDiff_LongRange = np.mean(diffLongRange)
meanDiff_OutRelRange = np.mean(diff_out_relError)



In [15]:
benchmark = 0
benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
print(baseline_deviation / (meanDiff_InRange**2) / 4)

benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
print(baseline_out_deviation / (meanDiff_OutRange**2) / 4)

benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
print(baseline_long_deviation / (meanDiff_LongRange**2) / 4)

benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
print(baseline_relError / (meanDiff_OutRelRange**2) / 4)

print(f"Benchmark: {benchmark}")

6.0861482165772385
0.23739514132280606
0.22541804948939323
0.12355976477918477
Benchmark: 6.672521172168623
